In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data_preprocessing import datetime_processing, userinput_processing, holiday_processing, create_x, create_y

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, accuracy_score

In [2]:
import os
os.chdir('/Users/varunvenkatesh/Documents/Github/ev_charging')
os.getcwd()

'/Users/varunvenkatesh/Documents/GitHub/ev_charging'

In [3]:
df = pd.read_parquet('data/sessions/caltech/').reset_index(drop=True)
df = datetime_processing(df)
df = userinput_processing(df)
df = holiday_processing(df)

In [4]:
df.head()

,_id,sessionID,stationID,spaceID,siteID,clusterID,connectionTime,disconnectTime,kWhDelivered,doneChargingTime,...,doneChargingTimeDay,userID,milesRequested,WhPerMile,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture,kWhRequested,is_holiday
0,5bc9160af9af8b0dad3c0690,2_39_95_27_2018-05-01 13:23:18.374684,2-39-95-27,CA-319,2,39,2018-05-01 13:23:18+00:00,2018-05-02 01:53:41+00:00,13.410,2018-05-01 17:17:07+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
1,5bc9160af9af8b0dad3c0691,2_39_79_380_2018-05-01 13:54:33.599590,2-39-79-380,CA-489,2,39,2018-05-01 13:54:34+00:00,2018-05-01 23:15:35+00:00,8.618,2018-05-01 15:35:39+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
2,5bc9160af9af8b0dad3c0692,2_39_139_28_2018-05-01 14:46:04.441389,2-39-139-28,CA-303,2,39,2018-05-01 14:46:04+00:00,2018-05-02 00:54:07+00:00,13.969,2018-05-01 23:57:02+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
3,5bc9160af9af8b0dad3c0693,2_39_79_378_2018-05-01 14:56:28.908623,2-39-79-378,CA-326,2,39,2018-05-01 14:56:29+00:00,2018-05-01 21:52:45+00:00,6.914,2018-05-01 16:07:56+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False
4,5bc9160af9af8b0dad3c0694,2_39_88_24_2018-05-01 15:14:50.443137,2-39-88-24,CA-314,2,39,2018-05-01 15:14:50+00:00,2018-05-01 22:51:30+00:00,13.785,2018-05-01 20:18:12+00:00,...,1.0,NaN,NaN,NaN,NaN,None,None,None,NaN,False


In [37]:
df.stationID.value_counts()

2-39-139-28     1792
2-39-131-30     1399
2-39-129-17     1136
2-39-81-4550    1126
2-39-89-25      1086
2-39-125-21     1042
2-39-127-19     1022
2-39-138-29      994
2-39-79-379      932
2-39-123-23      908
2-39-79-380      892
2-39-78-365      862
2-39-91-437      846
2-39-79-377      820
2-39-130-31      789
2-39-78-366      759
2-39-95-27       737
2-39-88-24       694
2-39-79-383      683
2-39-124-22      665
2-39-79-378      645
2-39-78-363      615
2-39-128-18      612
2-39-79-381      587
2-39-79-376      581
2-39-78-360      578
2-39-90-26       567
2-39-126-20      561
2-39-92-436      558
2-39-78-367      533
2-39-78-361      529
2-39-79-382      506
2-39-92-442      503
2-39-78-364      458
2-39-91-441      452
2-39-78-362      452
2-39-90-440      432
2-39-95-444      396
2-39-88-438      332
2-39-89-439      330
2-39-123-557     247
2-39-83-386      200
2-39-138-566     175
2-39-124-558     149
2-39-130-564     136
2-39-126-560     124
2-39-128-562     120
2-39-139-567 

In [5]:
def get_results(y_test, prediction):
    cm = confusion_matrix(y_test,prediction)
    make_classification_plot(cm)
    
    results = {'tpr': cm[1, 1]/np.sum(cm[1]),
               'fpr': cm[0,1]/np.sum(cm[0]),
               'accuracy': accuracy_score(y_test, prediction),
               'precision': precision_score(y_test, prediction),
               'recall': recall_score(y_test, prediction),
        'f1':f1_score(y_test,prediction)}
    return results

In [6]:
def make_classification_plot(cm):
    disp = ConfusionMatrixDisplay(cm)
    disp = disp.plot(include_values=True, cmap='viridis', ax=None, xticks_rotation='horizontal')
    plt.grid(False)
    plt.show()

In [7]:
results = {}

In [8]:
# create x and y 
start = '2018-05-01'
end = '2021-09-13'
spaceID = 'CA-303'
caiso_fp = 'data/caiso_dataset_5min.csv'

In [9]:
X = create_x(start, end, caiso_fp)
y = create_y(df, start, end, spaceID)

/Users/varunvenkatesh/Documents/GitHub/ev_charging/src/data_preprocessing.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  caiso_hourly = caiso.groupby(pd.Grouper(freq='1h')).mean()


In [10]:
#tmp = df.copy()
#tmp = tmp[tmp['spaceID'] == 'CA-303'].sort_index()

In [11]:
#y = pd.DataFrame(index=pd.date_range('2018-05-1','2021-09-13', inclusive='both', freq='h', tz=0),columns=['is_available','sessionID'])
#y['is_available'] = 1
#tmp.reset_index(inplace=True)
#for i in list(tmp.index):
#    start_ = tmp.loc[i,'connectionTime']
#    end_ = tmp.loc[i,'disconnectTime'] 
#    session_ = tmp.loc[i,'sessionID']
#    y.loc[start_:end_,['is_available','sessionID']] = 0, session_

In [12]:
#y.drop(columns='sessionID', inplace=True)

In [13]:
#y['is_available'].value_counts()

In [14]:
#caiso_df = pd.read_csv('data/caiso_dataset_5min.csv')
#caiso_df['datetime'] = pd.to_datetime(caiso_df['date'] + ' ' + caiso_df['Time'], utc=True)
#caiso_df.info()

In [15]:
#caiso_df = caiso_df.set_index('datetime')
#caiso_hourly = caiso_df.groupby(pd.Grouper(freq='1h')).mean()
#caiso_hourly.index.tz_localize(None)
#caiso_hourly.head()

In [16]:
#print(len(caiso_hourly))

In [17]:
#X = pd.DataFrame(index=pd.date_range('2018-05-1','2021-09-13', inclusive='both', freq='h', tz=0),columns=['dow','hour','month'])
#X = X.join(caiso_hourly)

In [18]:
X.head()

,dow,hour,month,is_holiday,Day ahead forecast,Hour ahead forecast,Current demand,Net demand,Net demand forecast,Solar,...,Geothermal CO2,Demand,Net Demand,Day-ahead demand forecast,Day-ahead net demand forecast,Resource adequacy capacity forecast,Net resource adequacy capacity forecast,Reserve requirement,Reserve requirement forecast,Resource adequacy credits
2018-05-01 00:00:00+00:00,1,0,5,False,21013.666667,21496.500000,21378.500000,17906.000000,NaN,0.0,...,9.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 01:00:00+00:00,1,1,5,False,20133.083333,20673.916667,20644.583333,17251.333333,NaN,0.0,...,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 02:00:00+00:00,1,2,5,False,19582.666667,20174.083333,20089.166667,16399.916667,NaN,0.0,...,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 03:00:00+00:00,1,3,5,False,19423.500000,19905.083333,19822.250000,15958.416667,NaN,0.0,...,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 04:00:00+00:00,1,4,5,False,19879.583333,20325.833333,20321.166667,16600.583333,NaN,0.0,...,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#X['dow'] = X.index.dayofweek
#X['hour'] = X.index.hour
#X['month'] = X.index.month
#X['connectionTime'] = X.index
#X = holiday_processing(X).drop(columns=['connectionTime'])
#X.head()

In [20]:
print(f'charger CA-303 is available {np.round(y.mean()*100,3)}% of the time')

charger CA-303 is available is_available    69.964
dtype: float64% of the time


/var/folders/7p/8d9trt0n3szgfv_nhh5rk6v00000gn/T/ipykernel_43218/1108395309.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(f'charger CA-303 is available {np.round(y.mean()*100,3)}% of the time')


In [21]:
X.fillna(0.0, inplace=True)

In [22]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29545 entries, 2018-05-01 00:00:00+00:00 to 2021-09-13 00:00:00+00:00
Freq: H
Data columns (total 39 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   dow                                      29545 non-null  int64  
 1   hour                                     29545 non-null  int64  
 2   month                                    29545 non-null  int64  
 3   is_holiday                               29545 non-null  bool   
 4   Day ahead forecast                       29545 non-null  float64
 5   Hour ahead forecast                      29545 non-null  float64
 6   Current demand                           29545 non-null  float64
 7   Net demand                               29545 non-null  float64
 8   Net demand forecast                      29545 non-null  float64
 9   Solar                                    29545 non-null  floa

In [23]:
X.shape[0] == y.shape[0]

True

# LSTM

In [24]:
# Create hold out test set
bin_numbers = pd.qcut(y['is_available'], q=10, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify=bin_numbers, random_state=42)
print(f'the training data has an average availability of {np.round(y_train.mean()*100,3)}%')

the training data has an average availability of is_available    69.652
dtype: float64%


/var/folders/7p/8d9trt0n3szgfv_nhh5rk6v00000gn/T/ipykernel_43218/2810031212.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(f'the training data has an average availability of {np.round(y_train.mean()*100,3)}%')


In [25]:
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
tf.random.set_seed(42)

2024-02-27 12:50:06.992272: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
#scaler = MinMaxScaler(feature_range=(0,1))
look_back = 1

In [27]:
model = Sequential()
model.add(LSTM(4, input_shape=(1,39)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

2024-02-27 12:50:11.215044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
X_train = np.reshape(X_train.values.astype(float), (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values.astype(float), (X_test.shape[0], 1, X_test.shape[1]))

In [29]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 1, 39) <dtype: 'float32'>
(None, 1) <dtype: 'float32'>
lstm (None, 1, 39) float32
dense (None, 4) float32


[None, None]

In [30]:
X_train.shape

(23636, 1, 39)

In [31]:
y_train = y_train.drop('sessionID', axis=1) 
y_test = y_test.drop('sessionID', axis=1)

In [32]:
model.fit(X_train, y_train, epochs=10, batch_size=8, verbose=2)

Epoch 1/10
2955/2955 - 7s - loss: 0.3650 - 7s/epoch - 3ms/step
Epoch 2/10
2955/2955 - 5s - loss: 0.2070 - 5s/epoch - 2ms/step
Epoch 3/10
2955/2955 - 4s - loss: 0.2104 - 4s/epoch - 1ms/step
Epoch 4/10
2955/2955 - 4s - loss: 0.2112 - 4s/epoch - 1ms/step
Epoch 5/10
2955/2955 - 4s - loss: 0.2111 - 4s/epoch - 1ms/step
Epoch 6/10
2955/2955 - 4s - loss: 0.2113 - 4s/epoch - 1ms/step
Epoch 7/10
2955/2955 - 4s - loss: 0.2112 - 4s/epoch - 1ms/step
Epoch 8/10
2955/2955 - 4s - loss: 0.2112 - 4s/epoch - 2ms/step
Epoch 9/10
2955/2955 - 5s - loss: 0.2112 - 5s/epoch - 2ms/step
Epoch 10/10
2955/2955 - 4s - loss: 0.2111 - 4s/epoch - 2ms/step


In [33]:
predictions = model.predict(X_test)

185/185 [==============================] - 1s 1ms/step


In [34]:
from sklearn.metrics import mean_squared_error

In [35]:
# MSE with CAISO
score = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {score}')

Mean Squared Error: 0.20553199554367607


# XGBoost

In [ ]:
from xgboost import XGBClassifier